## Solveur SAT

In [442]:
MAX_GRADE = 21

In [456]:
size = 100
nb_grades = 5
noise = 0
nb_class = 1

s = {i for i in range(nb_grades)}

s = frozenset(s)

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise, nb_class=nb_class)

grades, admissions = gen.generate_grades()

gen.analyze_gen()


if nb_class == 1:
    admissions = admissions.astype(int)


---------Analyze----------
Lambda: 0.29614569822486086
Weights: [0.30410878 0.25134193 0.00854258 0.21826292 0.21774379]
Betas: [14  9 14 12 10]
Got-in: {True: 61, False: 39}
% Got-in: {True: 61.0, False: 39.0} %
--------------------------


In [457]:
#Variable aplha
alpha = []
for i in range(nb_grades):
    for k in range(MAX_GRADE):
        alpha.append((i,k)) # ==> donne tous les alpha i k 

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


In [458]:
from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 


#Dictionnaire beta
v2i_beta = {frozenset(v) : A+i+1 for i,v in enumerate(subsets)}


In [459]:
#Clause 1
clause_1 = []

for i in range(nb_grades):
    for k in range(0,MAX_GRADE-1):
        for j in range(k+1,MAX_GRADE):
            clause_1.append([-v2i_alpha[(i,k)], v2i_alpha[(i,j)]])
         

In [460]:
#Clause 2
clause_2 = []

for i in subsets:
    C_prime = frozenset(i)
    for j in subsets:
        C = frozenset(j)
        if C.issubset(C_prime) and C != C_prime:
            clause_2.append([-v2i_beta[C], v2i_beta[C_prime]])



In [464]:
#Clause 3

clause_3 = []
for st_idx,student in enumerate(grades):
    if admissions[st_idx] == 1:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(v2i_alpha[(i,student[i])])
        
            clause_3.append(alpha+[v2i_beta[frozenset(s.difference(c))]])

clause_3

[[137],
 [19, 136],
 [25, 135],
 [61, 134],
 [76, 133],
 [93, 132],
 [19, 25, 131],
 [19, 61, 130],
 [19, 76, 129],
 [19, 93, 128],
 [25, 61, 127],
 [25, 76, 126],
 [25, 93, 125],
 [61, 76, 124],
 [61, 93, 123],
 [76, 93, 122],
 [19, 25, 61, 121],
 [19, 25, 76, 120],
 [19, 25, 93, 119],
 [19, 61, 76, 118],
 [19, 61, 93, 117],
 [19, 76, 93, 116],
 [25, 61, 76, 115],
 [25, 61, 93, 114],
 [25, 76, 93, 113],
 [61, 76, 93, 112],
 [19, 25, 61, 76, 111],
 [19, 25, 61, 93, 110],
 [19, 25, 76, 93, 109],
 [19, 61, 76, 93, 108],
 [25, 61, 76, 93, 107],
 [19, 25, 61, 76, 93, 106],
 [137],
 [8, 136],
 [27, 135],
 [49, 134],
 [77, 133],
 [99, 132],
 [8, 27, 131],
 [8, 49, 130],
 [8, 77, 129],
 [8, 99, 128],
 [27, 49, 127],
 [27, 77, 126],
 [27, 99, 125],
 [49, 77, 124],
 [49, 99, 123],
 [77, 99, 122],
 [8, 27, 49, 121],
 [8, 27, 77, 120],
 [8, 27, 99, 119],
 [8, 49, 77, 118],
 [8, 49, 99, 117],
 [8, 77, 99, 116],
 [27, 49, 77, 115],
 [27, 49, 99, 114],
 [27, 77, 99, 113],
 [49, 77, 99, 112],
 [8, 27

In [465]:
clause_4 = []


for st_idx,student in enumerate(grades):
    if admissions[st_idx] == 0:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(-v2i_alpha[(i,student[i])])
        
            clause_3.append(alpha+[-v2i_beta[frozenset(c)]])


In [466]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+1+A] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+A+1)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [467]:
#Lancer la résolution

myClauses= clause_1 + clause_2 + clause_3 + clause_4
myDimacs = clauses_to_dimacs(myClauses,len(i2v))

write_dimacs_file(myDimacs,"./workingfile.cnf")
res = exec_gophersat("./workingfile.cnf")

#Résultat
print(res)

(True, [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, 15, 16, 17, 18, 19, 20, 21, -22, -23, -24, -25, -26, -27, -28, -29, -30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, 76, 77, 78, 79, 80, 81, 82, 83, 84, -85, -86, -87, -88, -89, -90, -91, -92, -93, -94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, -106, -107, -108, -109, -110, -111, -112, 113, -114, -115, -116, -117, -118, -119, -120, -121, 122, -123, -124, 125, 126, -127, 128, 129, -130, 131, 132, 133, -134, 135, 136, 137], {(0, 0): False, (0, 1): False, (0, 2): False, (0, 3): False, (0, 4): False, (0, 5): False, (0, 6): False, (0, 7): False, (0, 8): False, (0, 9): False, (0, 10): False, (0, 11): False, (0, 12): False, (0, 13): False, (0, 14): True, (0, 15): True, (0, 16): True, (0, 17): True, (0, 18): True, (0, 19): True, (0, 20): True, (1, 0): False, (1, 1): False,

In [468]:
_, variables_idx, d = res

In [473]:
predicted = []
for student in grades:
    validated_courses = set()
    for i,k in enumerate(student):
        if d[(i,k)]:
            validated_courses.add(i)
    validated_courses = frozenset(validated_courses)
    if d[validated_courses]:
        predicted.append(1)
    else:
        predicted.append(0)
predicted
        


[1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1]

In [474]:
from sklearn.metrics import f1_score, accuracy_score

print(accuracy_score(admissions,predicted))
print(f1_score(admissions,predicted))

1.0
1.0


SAT Solver Multi classes

In [543]:
import sys
sys.path.append('../')

from generator import GradesGenerator

In [544]:
size = 100
nb_grades = 5
noise = 0
nb_class = 3

s = {i for i in range(nb_grades)}

s = frozenset(s)

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise, nb_class=nb_class)

grades, admissions = gen.generate_grades()


if nb_class == 1:
    admissions = admissions.astype(int)


In [545]:
#Variable aplha
alpha = []
for i in range(nb_grades):
    for k in range(MAX_GRADE):
        for h in range(nb_class+1):
            alpha.append((i,k,h)) # ==> donne tous les alpha i k h

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


In [546]:
from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 


#Dictionnaire beta
v2i_beta = {frozenset(v) : A+i+1 for i,v in enumerate(subsets)}


In [547]:
clause_1 = []

for i in range(nb_grades):
    for k in range(MAX_GRADE-1):
        for h in range(nb_class):
            for j in range(k+1,MAX_GRADE):
                clause_1.append([-v2i_alpha[(i,k,h)], v2i_alpha[(i,j,h)]])

In [548]:
clause_2 = []

for i in subsets:
    C_prime = frozenset(i)
    for j in subsets:
        C = frozenset(j)
        if C.issubset(C_prime) and C != C_prime:
            clause_2.append([-v2i_beta[C], v2i_beta[C_prime]])


In [549]:
clause_3 = []


clause_3 = []
for st_idx,student in enumerate(grades):
    for c in subsets:
        alpha = []
        for i in c:
            alpha.append(v2i_alpha[(i,student[i],admissions[st_idx])])
    
        clause_3.append(alpha+[v2i_beta[frozenset(s.difference(c))]])





In [550]:
clause_4 = []

for st_idx,student in enumerate(grades):
    if admissions[st_idx] < nb_class:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(-v2i_alpha[(i,student[i],admissions[st_idx]+1)])
        
            clause_4.append(alpha+[-v2i_beta[frozenset(c)]])


In [551]:
clause_5 = []

for k in range(MAX_GRADE):
    for i in range(nb_grades):
        for h in range(nb_class):
            for j in range(h+1, nb_class+1):
                clause_5.append([v2i_alpha[(i,k,h)],-v2i_alpha[(i,k,j)]])



In [552]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+A+1] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+1+A)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [553]:
myClauses= clause_1 + clause_2 + clause_3 + clause_4 + clause_5
myDimacs = clauses_to_dimacs(myClauses,len(i2v))

write_dimacs_file(myDimacs,"./workingfile_multi.cnf")
res = exec_gophersat("./workingfile_multi.cnf")

#Résultat
print(res)

(True, [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, 25, -26, -27, -28, 29, -30, -31, -32, 33, -34, -35, -36, 37, -38, -39, -40, 41, 42, 43, -44, 45, 46, 47, -48, 49, 50, 51, -52, 53, 54, 55, 56, 57, 58, 59, -60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, -72, 73, 74, 75, -76, 77, 78, 79, -80, 81, 82, 83, -84, 85, -86, -87, -88, 89, -90, -91, -92, 93, -94, -95, -96, 97, -98, -99, -100, 101, -102, -103, -104, 105, -106, -107, -108, 109, -110, -111, -112, 113, -114, -115, -116, 117, -118, -119, -120, 121, -122, -123, -124, 125, 126, -127, -128, 129, 130, -131, -132, 133, 134, -135, -136, 137, 138, -139, -140, 141, 142, 143, -144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, -160, 161, 162, 163, 164, 165, 166, 167, -168, 169, -170, -171, -172, 173, -174, -175, -176, 177, -178, -179, -180, 181, -182, -183, -184, 185, -186, -187, -188, 189, -190, -191, -192, 193, -194, -195, -196, 197, -198, -19

In [554]:
_, variables_idx, d = res

In [555]:
predicted = []
for student in grades:
    current_class = 0
    for h in range(nb_class+1):
        current_class = h
        validated_courses = set()
        for i,k in enumerate(student):
            if d[(i,k,h)]:
                validated_courses.add(i)
        validated_courses = frozenset(validated_courses)
        if validated_courses in d.keys():
            if d[validated_courses]:
                continue
            else:
                break
        else:
            break
    predicted.append(current_class)
        


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]